In [ ]:
pip install wandb

In [10]:
import torch
import torchvision
import wandb

In [20]:
wandb.init(project="fashion mnist")

In [21]:
class my_model(torch.nn.Module):
  def __init__(self):
    super().__init__()
    # self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding_mode='same')
    # self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding_mode='same')
    self.fc1 = torch.nn.Linear(784, 256)
    self.fc2 = torch.nn.Linear(256, 128)
    self.fc3 = torch.nn.Linear(128, 64)
    self.fc4 = torch.nn.Linear(64, 10)

  def forward(self, x):
    x = x.reshape((x.shape[0], 784))

    x = self.fc1(x)
    x = torch.relu(x)
    x = torch.dropout(x, 0.2, train=True)

    x = self.fc2(x)
    x = torch.relu(x)
    x = torch.dropout(x, 0.2, train=True)

    x = self.fc3(x)
    x = torch.relu(x)
    x = torch.dropout(x, 0.2, train=True)

    x = self.fc4(x)
    x = torch.softmax(x, dim=1)
    return x

In [22]:
device = torch.device("cuda")
# device = torch.device("cpu")

model = my_model()
model = model.to(device)
model.train(True)

my_model(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)

In [23]:
# parameters

batch_size = 64
epochs = 10
config = wandb.config
config.learning_rate = 0.001

In [24]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0), (1)),
    torchvision.transforms.RandomHorizontalFlip(0.5),
    # torchvision.transforms.Scale()
])

dataset = torchvision.datasets.FashionMNIST("./dataset", train=True, download=True, transform=transform)
train_data = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
loss_function = torch.nn.CrossEntropyLoss()

In [26]:
def calc_acc(preds, labels):
  _, pred = torch.max(preds, 1)
  acc = torch.sum(pred == labels.data, dtype=torch.float64) / len(preds)
  return acc

In [27]:
for epoch in range(epochs):
  train_loss = 0.0
  train_acc = 0.0
  for images, labels in train_data:
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()

    preds = model(images)

    loss = loss_function(preds, labels)
    loss.backward()

    optimizer.step()

    train_loss += loss
    train_acc += calc_acc(preds, labels)

  total_loss = train_loss / len(train_data)
  total_acc = train_acc / len(train_data)
  print(f"Epoch: {epoch+1}, Loss: {total_loss}, Accuracy: {total_acc}")

  wandb.log({'epochs':  epoch + 1,
              'loss': total_loss,
              'acc': total_acc
                              })

Epoch: 1, Loss: 1.7543165683746338, Accuracy: 0.7120535714285714
Epoch: 2, Loss: 1.661380648612976, Accuracy: 0.8015724946695095
Epoch: 3, Loss: 1.6468504667282104, Accuracy: 0.814215751599147
Epoch: 4, Loss: 1.637058138847351, Accuracy: 0.8240271855010661
Epoch: 5, Loss: 1.631467580795288, Accuracy: 0.8294576226012793
Epoch: 6, Loss: 1.6300042867660522, Accuracy: 0.8302738539445629
Epoch: 7, Loss: 1.6267271041870117, Accuracy: 0.8338219616204691
Epoch: 8, Loss: 1.623963713645935, Accuracy: 0.8365371801705757
Epoch: 9, Loss: 1.6198234558105469, Accuracy: 0.8406183368869936
Epoch: 10, Loss: 1.621852993965149, Accuracy: 0.8387526652452025


In [28]:
torch.save(model.state_dict(), "fashion_mnist.pth")

In [ ]:
import cv2
import numpy as np

# model.tarin(False)
my_model.eval()

# preprocess
img = cv2.imread('5.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))

tensor = transform(img).unsqueeze(0).to(device)
# tensor = tensor.to(device)

# process
preds = my_model(tensor)

# postprocess
preds = preds.cpu().detach().numpy()
output = np.argmax(preds)
output